# Basic MLP experimentation

In [33]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from typing import List

torch.manual_seed(101)

In [8]:
Transform = transforms.ToTensor()
train = datasets.MNIST(root="../", train=True, download=False, transform=Transform)
test = datasets.MNIST(root="../", train=False, download=False, transform=Transform)

In [76]:
TRAIN_BATCH_SIZE = 100
TEST_BATCH_SIZE = 500
train_loader = DataLoader(train, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test, batch_size=TEST_BATCH_SIZE, shuffle=True)

In [77]:
class MLP(nn.Module):
    def __init__(self, input_size: int, output_size: int, layer_sizes: List[int]):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.layers = [
            nn.Linear(self.input_size, layer_sizes[0]),
            nn.ReLU(),
        ]
        for i in range(1, len(layer_sizes)):
            previous_layer_size = layer_sizes[i-1]
            current_layer_size = layer_sizes[i]
            new_layer = nn.Linear(previous_layer_size, current_layer_size)
            self.layers.append(new_layer)
            self.layers.append(nn.ReLU())
        
        output_layer = nn.Linear(layer_sizes[-1], output_size)
        self.layers.append(output_layer)
        self.layers.append(nn.LogSoftmax(dim=1))
        self.model = nn.Sequential(*self.layers)
        
    def forward(self, x):
        return self.model(x)

In [78]:
model = MLP(
    input_size = 784,
    output_size = 10, 
    layer_sizes = [100, 100]
)

In [79]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [83]:
EPOCHS = 10
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for epoch in range(EPOCHS):
    epoch_train_accurate = 0
    epoch_test_accurate = 0
    for i, (image, label) in enumerate(train_loader):
        prediction_vector = model(image.view(TRAIN_BATCH_SIZE, -1))
        loss = criterion(prediction_vector, label)
        predicted_class = torch.max(prediction_vector, 1)[1]
        batch_accurate = (predicted_class == label).sum()
        epoch_train_accurate += batch_accurate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 200 == 199:
            print(
                f"""epoch: {epoch+1}/{EPOCHS}, 
                batch: {i+1}, 
                train loss: {loss.item()}, 
                train accuracy: {epoch_train_accurate/((i+1) * TRAIN_BATCH_SIZE)}"""
            )
    train_losses.append(loss)
    train_correct.append(epoch_train_accurate)
    with torch.no_grad():
        for i, (image, label) in enumerate(test_loader):
            prediction_vector = model(image.view(TEST_BATCH_SIZE, -1))
            predicted_class = torch.max(prediction_vector.data, 1)[1]
            epoch_test_accurate += (predicted_class == label).sum()
    test_correct.append(epoch_test_accurate)
    print(f"TEST STEP for epoch {epoch+1} || accuracy: {epoch_test_accurate/10000}")

epoch: 1/10, 
                batch: 200, 
                train loss: 0.09538207948207855, 
                train accuracy: 0.9718000292778015
epoch: 1/10, 
                batch: 400, 
                train loss: 0.04335453361272812, 
                train accuracy: 0.9724000096321106
epoch: 1/10, 
                batch: 600, 
                train loss: 0.0526336245238781, 
                train accuracy: 0.9730499982833862
TEST STEP for epoch 20 || accuracy: 0.9702000021934509
epoch: 2/10, 
                batch: 200, 
                train loss: 0.1058955118060112, 
                train accuracy: 0.9772999882698059
epoch: 2/10, 
                batch: 400, 
                train loss: 0.11305034905672073, 
                train accuracy: 0.9779000282287598
epoch: 2/10, 
                batch: 600, 
                train loss: 0.09886480122804642, 
                train accuracy: 0.9777166843414307
TEST STEP for epoch 20 || accuracy: 0.975600004196167
epoch: 3/10, 
               